In [2]:
import os
import json
from mappings import classToAnsible, ansibleToClass # need to get the mapping to ansible from json
import yaml
import pprint
from isConfigurableMap import isConfigurableMap # need to get the 
from collections import Counter
from collections import deque
from defaultClassAttrValues import defaults
from exceptionDict import exceptions
from requiredParamsAliasesMap import requiredParamsAliases, reverse_alias_map
from paramAliasMap import paramAliasMap

In [3]:
# load the json file

with open('tn-nils-4.json', 'r') as file:
    data = json.load(file)

In [4]:
# this is mostly chatGPT, will need to rewrite it and figure out how to do this without it

names = []

def prune_dict(data):
    def contains_name_key(d):
        # Check if 'name' key is in the current dictionary
        if 'name' in d:
            return True
        # Check nested dictionaries
        for key, value in d.items():
            if isinstance(value, dict) and contains_name_key(value):
                return True
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict) and contains_name_key(item):
                        return True
        return False

    def prune(d):
        keys_to_delete = []
        for key, value in d.items():
            if isinstance(value, dict):
                if not contains_name_key(value):
                    keys_to_delete.append(key)
                else:
                    prune(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        prune(item)

        for key in keys_to_delete:
            del d[key]

    if isinstance(data, dict):
        prune(data)
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                prune(item)

# maybe only modules appear in the playbooks, not classes (?)

In [5]:
def map_isConfigurable_to_ansible(data):
    for key, value in data.items():
        map_json_to_ansible(data, key)

In [6]:
with open('aci-meta-5.3-2c.json', 'r') as file:
    meta = json.load(file)

In [7]:
import os
import re
import pprint

PATH_TO_MODULE_DIR = "/Users/lpaggen/Documents/research_project/ansible-aci/modules"

def get_file_paths(dir):
    py_files = []
    for file in os.listdir(dir):
        file_path = os.path.join(dir, file)
        if file.endswith(".py"):
            py_files.append(file_path)
    return py_files

# code is bad at the moment, will address if needed, for the time being it runs well enough

def extract_parameter_aliases(dir):
    count = 0

    py_files = get_file_paths(dir)
    out = {}

    # regex 
    # finds the block of code in source code after "argument_spec.update()"
    pattern = re.compile(r'argument_spec\.update\(\s*(.*?)^\s*\)', re.DOTALL | re.MULTILINE)
    pattern2 = re.compile(r"([a-zA-Z_]+)\s*=\s*dict", re.MULTILINE)
    pattern3 = re.compile(r"([a-zA-Z_]+)\s*=\s*dict\s*\((.*?)\)", re.DOTALL)
    
    # finds the aliases of the params according to what is found in match 3
    pattern4 = re.compile(r'(?<=aliases=\[)([^\]]*)', re.DOTALL | re.MULTILINE)
    
    # finds the block of code in source after "class_config=dict()"
    pattern5 = re.compile(r'class_config\s*=\s*dict\s*\(\s*([^)]+)', re.DOTALL)
    
    # finds the kw in the pattern5
    pattern6 = re.compile(r"([^=,\s]+)=([^'\n,]*)")

    for file_path in py_files:
        with open(file_path, 'r') as file:
            file_content = file.read()

        matches = pattern.findall(file_content)
        match5 = pattern5.findall(file_content)

        entry_dict = {}  # nested dict

        for entries_str in matches:
            match2 = pattern2.findall(entries_str) # list of the parameters
            match3 = pattern3.findall(entries_str)

            if match3:
                for idx, i in enumerate(match3): # get idx to find correspondance in match2
                    match4 = pattern4.findall(str(i)) # need string otherwise error sometimes
                    if match4:
                        for j in match4:
                            entry_dict[match2[idx]] = [i.replace("\"", "").replace(" ", "") for i in j.split(",")] # clean string
                    else:
                        entry_dict[match2[idx]] = None

            # now this looks for the second set of aliases (source code seems to be weird)
            # it should overwrite the "None" values if any
            # if match5:
            #     for idx, i in enumerate(match5):
            #         temp = i.split(",")
            #         # print(file_path.split("/")[-1][:-3])
            #         for j in temp:
            #             match6 = pattern6.search(str(j))
            #             if match6:
            #                 # print(match6.group(2), match6.group(1))
            #                 try:
            #                     if entry_dict[match6.group(2)] == None:
            #                         entry_dict[match6.group(2)] = [match6.group(1)] # ",".join([i for i in match6.group(1)])
            #                     elif match6.group(2) != None and type(match6.group(2)) != str: # error somewhere
            #                         entry_dict[match6.group(2)].append(match6.group(1))
            #                 except(KeyError):
            #                     entry_dict[match6.group(2)] = [match6.group(1)]

                print("\n")

        out[file_path.split("/")[-1][:-3]] = entry_dict

    return out

In [8]:
# this parser just gets the rnFormat of each class

def getClassRnFormat(data):
    out = {}

    def process_data(data, parent_key = None):
        if isinstance(data, dict):
            for key, value in data.items():

                if key == "rnFormat":
                    try:
                        entry_key = classToAnsible[parent_key]
                    except(KeyError):
                        entry_key = parent_key

                    out[entry_key] = value if len(value) > 0 else None

                if isinstance(value, (dict, list)):
                    process_data(value, key)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process_data(item, parent_key)

    process_data(data)

    return out

In [9]:
a = getClassRnFormat(meta)

In [10]:
# save to file
with open("classRnFormat.py", 'w') as file:
    file.write("rnFormat = ")
    pprint.pprint(a, stream=file)

print(f"Dictionary has been saved to classRnFormat.py")

Dictionary has been saved to classRnFormat.py


# need way to decide which mapping, according to the parent in the json

In [11]:
exception_list=['spanDestGrp',
                'spanSrcGrp',
                'spanSrc',
                'mgmtMaintP',
                'spanRsSrcToPathEp',
                'fvSubnet',
                'fvRsPathAtt',
                'vzBrCP',
                'dhcpRelayP',
                'infraRsVlanNs',
                'fvRsSecInherited',
                'l1PhysIf',
                'l2extInstP',
                'mgmtOoB']

In [12]:
Counter(ansibleToClass.values())

Counter({'spanDestGrp': 3,
         'spanSrcGrp': 3,
         'spanSrc': 3,
         'mgmtMaintP': 3,
         'spanRsSrcToPathEp': 2,
         'fvSubnet': 2,
         'fvRsPathAtt': 2,
         'vzBrCP': 2,
         'dhcpRelayP': 2,
         'infraRsVlanNs': 2,
         'fvRsSecInherited': 2,
         'l1PhysIf': 2,
         'l2extInstP': 2,
         'mgmtOoB': 2,
         'leakInternalSubnet': 1,
         'leakRoutes': 1,
         'leakTo': 1,
         'aaaRbacClassPriv': 1,
         'aaaDomain': 1,
         'aaaRole': 1,
         'aaaSshAuth': 1,
         'aaaUser': 1,
         'aaaUserCert': 1,
         'aaaUserDomain': 1,
         'aaaUserRole': 1,
         'infraHPortS, infraPortBlk': 1,
         'infraHPortS, infraRsAccBaseGrp, infraPortBlk': 1,
         'spanFilterGrp': 1,
         'spanFilterEntry': 1,
         'infraHPortS, infraSubPortBlk': 1,
         'infraAttEntityP, infraProvAcc': 1,
         'infraRsDomP': 1,
         'infraRsFuncToEpg': 1,
         'fvAp': 1,
         

In [13]:
from defaultClassAttrValues import defaults

In [14]:
reverseAliases = reverse_alias_map(requiredParamsAliases)

# TO DO 

## add a list which can hold all children dictionaries somehow, NEEDED to rebuild the final yml

In [15]:
def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', "", "::", ":all:"]

    # invisible_arguments
    invisible_args = ["annotation", "dn"] # adjust as needed

    # define exception list
    exception_list = ['aci_access_span_src_group',
                        'aci_bd',
                        'aci_bd_dhcp_label',
                        'aci_bd_subnet',
                        'aci_domain_to_vlan_pool',
                        'aci_epg_subnet',
                        'aci_fabric_span_src_group',
                        'aci_tenant_span_src_group',
                        'aci_tenant_span_src_group_src']
    
    # here define duplicates list - handle dupes in values of class mappings
    # key (class) : value (dict) -> key (parent class) : value (correct mapping)

    duplicate_list = ['spanDestGrp', 'spanSrcGrp', 'spanSrc',
                      'mgmtMaintP', 'spanRsSrcToPathEp', 'fvSubnet',
                      'fvRsPathAtt', 'vzBrCP', 'dhcpRelayP',
                      'infraRsVlanNs', 'fvRsSecInherited', 'l1PhysIf',
                      'l2extInstP', 'mgmtOoB']

    # TO DO
    # finish this mapping
    duplicate_map = {
        # 'spanDestGrp': None,
        # 'spanSrcGrp': None,
        # 'spanSrc': None,
        # 'mgmtMaintP': None,
        # 'spanRsSrcToPathEp': None,
        'fvSubnet': {'aci_epg': 'aci_epg_subnet',
                     'aci_bd': 'aci_bd_subnet'},
        # 'fvRsPathAtt': None,
        'vzBrCP': "aci_contract",
        # 'dhcpRelayP': None,
        # 'infraRsVlanNs': None,
        # 'fvRsSecInherited': None,
        # 'l1PhysIf': None,
        # 'l2extInstP': None,
        # 'mgmtOoB': None
        }
    
    def map_json_to_ansible(json_data, key, map):
        try:
            new_key = map[key]
            json_data[new_key] = json_data.pop(key) # replace with ansible term
        except(KeyError): # no match found, skip
            pass

    def remove_isNotConfigurable(key, delete_key_list):
        try:
            if isConfigurableMap[key] == False:
                delete_key_list.append(key)
            else:
                pass
        except(KeyError):
            pass

    # helper method to navigate the exception dictionary
    def map_if_duplicate(child, parent = None):
        out = None
        try:
            out = duplicate_map[child][parent]
        except(KeyError):
            out = duplicate_map[child] # means there is no exception with the parent
        return out

    # simple method to check if a key in in the exception list
    # need to run this stuff before mapping anything - mapping is ambiguous
    def isduplicate(key):
        return key in duplicate_list

    def isdefault(parent_key, key, value, map):
        try:
            return value == map[parent_key][key] # means attribute has a default value
        except(KeyError):
            return False
        
    def isexception(key): # checks if key in exceptions
        try:
            return classToAnsible[key] in exception_list
        except(KeyError):
            return False

    def isfullydefault(val, parent_key, default_map, second_default_map):
        try:
            # convert defaults to sets 
            default_values_set = set(default_map[parent_key].values())
            second_default_values_set = set(second_default_map)
            
            # combine default args and default mapping
            combined_default_values_set = default_values_set.union(second_default_values_set)
            
            # to set 
            val_set = set(val)
            
            # check if subset
            return val_set.issubset(combined_default_values_set)
        except KeyError: # anything which is not "fv" is here atm
            return True # change based on desired behavior
        
    def save_to_yaml(save_path, data):
        # save_path = os.path.join(save_path, "ansible_reconstructed.yml")
        with open(save_path, 'w') as file:
            yaml.dump(data, file, default_flow_style=False)
        print(f"YAML file has been saved to {save_path}")

    # process data recursively
    # we need to track many parent-child keys, including some of sublists etc
    def process(data, parent_key:str = None, children_parent_key:str = None, attribute_parent_key:str = None, default_map:dict = defaults, dn:dict = {}, subclasses = []) -> dict:
        """

        Handles mapping of 
        - classes to ansible
        - class parameters to ansible

        Gets rid of the "attributes" field found in the JSON config 

        Gets rid of any fields with default values

        Handles duplicates in mappings

        """

        changes = []

        yml_tasks = [] # we will also need a list of the nested dictionaries for yml correct format

        if isinstance(data, dict):
            keys_to_delete = [] # maintain this list outside any loop, prevents "changed size" errors

            for key, value in data.items():

                # handle dn - name of parent in child, grandchild etc etc
                if isinstance(value, dict) and "children" in value.keys():
                    try:
                        dn_key = reverseAliases[classToAnsible[key]]["name"][:-1]
                        dn_val = data[key]['attributes']["name"]
                        dn[dn_key] = dn_val
                    except(KeyError):
                        pass

                # all CLASSES are handled, exceptions..
                # TO DO -> handle exceptions like "changes"
                # also not quite sure everything we are looking for is exclusively under "children" ...
                if key == "children":
                    children_parent_key = parent_key

                    # children ALWAYS contains a nested list of nested dictionaries
                    for item in value:
                        for chld_key, chld_value in item.items():

                            # the last element of each tuple in "changes" is an integer
                            # this is ONLY TO IDENTIFY WHICH TYPE OF CHANGE IS NEEDED

                            # handle duplicate mappings FIRST
                            if isduplicate(chld_key):
                                changes.append((children_parent_key, chld_key, chld_value, 0))

                # all PARAMETERS are processed and mapped if needed
                elif key == "attributes": # these are all the parameters of the classes
                    attribute_parent_key = parent_key

                    # check if all attributes are default; if so, skip processing entirely
                    # empty dictionaries function will take care of it
                    if not isfullydefault(value.values(), parent_key, defaults, default_args):

                        if isexception(parent_key):

                            # TYPE 1 EXCEPTIONS -> ADD PARAMS WHICH ARE FOUND IN SUBCLASSES (handled same as type2 actually)
                            try:
                                for param, path in exceptions[classToAnsible[parent_key]].items():
                                    for item in data['children']: # it seems "children" is almost always found here
                                        if path[0] in item.keys():
                                            changes.append((parent_key, param, item[path[0]][path[1]][path[2]], 2))

                                            # append the class we will "throw away"
                                            subclasses.append(path[0])
                                            print(subclasses)

                                            #######
                                            # path[0] is the exception class we need to get rid of, maybe through the changes system
                                            #######

                            except(KeyError) as e:
                                print(f"KeyError encountered: {e}")
                                pass

                        # TYPE 2 -> fix the dn attributes missing from children classes
                        if len(dn) > 0:
                            # assuming dn can only be a stack 2 long
                            if len(dn) > 2:
                                dn_list = list(dn.items())
                                dn_list.pop(1)
                                dn = dict(dn_list)
                            for dn_key, dn_val in dn.items():
                                changes.append((attribute_parent_key, dn_key, dn_val, 2))

                        # TYPE 2 CHANGES -> ATTRIBUTES FIELD REMOVAL, REMOVAL OF DEFAULTS
                        # append all non default params to "changes" - mapping and deletion is done here
                        for attr_key, attr_value in value.items(): # here empty values are also handled, eg if default then don't change
                            if attr_key not in invisible_args and attr_value not in default_args and not isdefault(attribute_parent_key, attr_key, attr_value, default_map):

                                # proper to fv_subnet once again?
                                # new exception found with "ip" -> creates a mask
                                if attr_key == "ip":
                                    attr_key = "gateway"
                                    changes.append((attribute_parent_key, 'mask', int(attr_value[-2:]), 2))

                                attr_value = attr_value if attr_key != "gateway" else attr_value[:-3]

                                changes.append((attribute_parent_key, attr_key, attr_value, 2)) # appends a tuple

                        # atm hardcoding "present" into yml, change later if needed
                        changes.append((attribute_parent_key, 'state', 'present', 2))

                    keys_to_delete.append("attributes") # can append key too

                # any exception will be caught here and will crash the entire program
                # therefore we want no exceptions
                if isinstance(value, (dict, list)):
                    try:
                        process(value, key, attribute_parent_key, dn)
                    except (KeyError) as e:
                        print(data['mldSnoopPol'])
                        print(f"Error processing key {key}: {e}")
                        pass

                if isinstance(value, dict) and not value:
                    keys_to_delete.append(key)

                remove_isNotConfigurable(key, keys_to_delete) # change to boolean check? 

            # any modification to the dict can only be made outside the loops
            for key in keys_to_delete:
                try:
                    del data[key]
                except(KeyError):
                    pass

            # use the mapping function
            for key in list(data.keys()):
                if key not in duplicate_list: # mapping done elsewhere for dupes
                    map_json_to_ansible(data, key, classToAnsible)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process(item, parent_key, attribute_parent_key, dn)

        # accessing keys in "data" instead of passing through a parent_key works
        # BECAUSE OF RECURSION, we are handling nested dictionaries as "data" everytime !!!!!!
        for change in changes:
            parent_key, child_key, child_value, change_type = change  # "change" is a 3x tuple
            if change_type == 2: # ATTRIBUTES parent key
                try:
                    new_key = reverseAliases[classToAnsible[parent_key]][child_key]
                    # at the moment no use for the required tag, can change later on if we need it somehow
                    if new_key[-1] == "*":
                        new_key = new_key[:-1]

                    data[new_key] = child_value
                    if child_key in data and new_key != child_key: # UNSURE if this is necessary anymore ..
                        del data[child_key]
                except KeyError:
                    data[child_key] = child_value

            # DOUBLE CHECK logic here, otherwise seems to work as expected
            # also can get rid of the whole "CHILDREN" key and move all upwards when we need it, only issue is how does this translate in yml
            # FOR NOW keep "children" key as it helps with ordering everything correctly
            elif change_type == 0: # CHILDREN parent key, DUPLICATE CASES
                try:
                    new_key = map_if_duplicate(child_key, classToAnsible[parent_key])
                    data['children'][0][new_key] = child_value
                    if child_key in data['children'][0]: # is this always the case? 0 idx? double check logic
                        del data['children'][0][child_key]
                except(KeyError):
                    pass

        return data

    # this function gets rid of empty structures in the output, think of {} for example
    # code is from GPT
    def remove_empty_dicts(data):
        if isinstance(data, dict):
            return {k: remove_empty_dicts(v) for k, v in data.items() if remove_empty_dicts(v)} # see return of function for why this works
        elif isinstance(data, list):
            for idx, item in enumerate(data):
                if isinstance(item, dict) and len(item) == 0:
                    data.pop(idx)
                else:
                    data[idx] = remove_empty_dicts(item)
            return data
        else:
            return data # this basically evaluates to false when the structure is None

    def rebuild_yml(data, credentials_file = None):
        if isinstance(data, dict):
            for key, value in data.items():
                print(key)

                if isinstance(value, dict):
                    rebuild_yml(value)
                elif isinstance(value, list):
                    for item in value:
                        rebuild_yml(item)
        elif isinstance(data, list):
            for item in data:
                rebuild_yml(item)

    # handle reconstruction of yml with recursion
    def rebuild_yml(data, credentials_file = None, yml_list = [], parent_key = None):

        entry_dict = {}

        if isinstance(data, dict):
            for key, value in data.items():
                nested_dictionary = {}

                # here restructuring is handled
                if parent_key == "children" or parent_key == None:
                    entry_dict["name"] = "THIS IS NOT SET YET"
                    for subkey, subvalue in data[key].items():
                        if subkey != "children":
                            # print(key, subkey, subvalue)
                            nested_dictionary[subkey] = subvalue
                    entry_dict["cisco.aci." + key] = nested_dictionary
                    entry_dict["delegate_to"] = "localhost"

                    yml_list.append(entry_dict)

                if isinstance(value, dict):
                    rebuild_yml(value, parent_key = key)

                elif isinstance(value, list):
                    for item in value:
                        rebuild_yml(item, parent_key = key)

        elif isinstance(data, list):
            for item in data:
                rebuild_yml(item, parent_key = key)

        return yml_list

    # process the initial data
    a = process(data)

    # delete the empty dictionaries
    b = remove_empty_dicts(a)

    save_to_yaml("ansible_reconstructed.yml", data)

    return data

In [16]:
with open("tn-nils-5.json", 'r') as file:
    y = json.load(file)

In [17]:
%%time
test = reconstruct_yml(y)

['fvRsBdToEpRet']
['fvRsBdToEpRet', 'fvRsBdToEpRet']
['fvRsBdToEpRet', 'fvRsBdToEpRet', 'fvRsIgmpsn']
['fvRsBdToEpRet', 'fvRsBdToEpRet', 'fvRsIgmpsn', 'fvRsBDToNdP']
['fvRsBdToEpRet', 'fvRsBdToEpRet', 'fvRsIgmpsn', 'fvRsBDToNdP', 'fvRsMldsn']
['fvRsBdToEpRet', 'fvRsBdToEpRet', 'fvRsIgmpsn', 'fvRsBDToNdP', 'fvRsMldsn', 'fvRsCtx']
KeyError encountered: 'children'
YAML file has been saved to ansible_reconstructed.yml
CPU times: total: 46.9 ms
Wall time: 61.9 ms


In [18]:
test

{'aci_tenant': {'children': [{'fvRsTenantMonPol': {'tenant': 'nils',
     'name': 'bobaf',
     'modTs': '2024-07-05T00:55:16.386+00:00',
     'monPolDn': 'uni/tn-common/monepg-default',
     'rn': 'rsTenantMonPol',
     'state': 'present',
     'stateQual': 'default-target',
     'tDn': 'uni/tn-common/monepg-default',
     'tRn': 'monepg-default',
     'uid': '0'}},
   {'fvEpTags': {'tenant': 'nils',
     'name': 'bobaf',
     'modTs': '2024-07-05T00:55:16.386+00:00',
     'monPolDn': 'uni/tn-common/monepg-default',
     'rn': 'eptags',
     'uid': '0',
     'state': 'present'}},
   {'aci_tenant_ep_retention_policy': {'tenant': 'nils',
     'epr_policy': 'mangaboul',
     'modTs': '2024-07-16T02:20:56.723+00:00',
     'rn': 'epRPol-mangaboul',
     'uid': '15374',
     'state': 'present'}},
   {'aci_vrf': {'children': [{'fvRsVrfValidationPol': {'tenant': 'nils',
        'epr_policy': 'mangaboul',
        'vrf': 'bob',
        'modTs': '2024-07-05T00:55:19.652+00:00',
        'monPolDn

In [19]:
# handle reconstruction of yml with recursion

# TO DO -> ADD REMVOVAL OF "fvRs" CLASSES, too hard to handle above due to recursive issues

def rebuild_yml(data, credentials_file = None, yml_list = [], parent_key = None, grandparent_key = None):

    entry_dict = {}

    if isinstance(data, dict):
        for key, value in data.items():

            nested_dictionary = {}

            # here restructuring is handled
            if parent_key == "children" or parent_key == None:

                # bad trick to get rid of all non-aci modules
                if key[:3] == "aci":
                    pass
                else:
                    break

                # set name to something generic
                key_name = key.split("_")[1] if len(key.split("_")) == 2 else key.split("_")[2]
                if parent_key != None:
                    grandparent_key_name = grandparent_key.split("_")[1] if len(grandparent_key.split("_")) == 2 else grandparent_key.split("_")[2:]
                    sentence = f"add a {key_name.upper()} to the {grandparent_key_name.upper()}"
                else:
                    sentence = f"create {key_name} on ACI using {key} module"

                entry_dict["name"] = sentence
                for subkey, subvalue in data[key].items():
                    if subkey != "children":
                        # print(key, subkey, subvalue)
                        nested_dictionary[subkey] = subvalue
                entry_dict["cisco.aci." + key] = nested_dictionary
                entry_dict["delegate_to"] = "localhost"

                yml_list.append(entry_dict)

            if isinstance(value, dict):
                rebuild_yml(value, parent_key = key, grandparent_key = parent_key)

            elif isinstance(value, list):
                for item in value:
                    rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

    elif isinstance(data, list):
        for item in data:
            rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

    return yml_list

a = rebuild_yml(test)

with open("ansible_reconstructed.yml", 'w') as file:
    yaml.dump(a, file, default_flow_style = False, sort_keys = False)
print(f"YAML file has been saved to ansible_reconstructed.yml")

YAML file has been saved to ansible_reconstructed.yml


In [20]:
a

[{'name': 'create tenant on ACI using aci_tenant module',
  'cisco.aci.aci_tenant': {'tenant': 'nils',
   'modTs': '2024-07-05T00:55:16.386+00:00',
   'monPolDn': 'uni/tn-common/monepg-default',
   'uid': '15374',
   'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a EP to the TENANT',
  'cisco.aci.aci_tenant_ep_retention_policy': {'tenant': 'nils',
   'epr_policy': 'mangaboul',
   'modTs': '2024-07-16T02:20:56.723+00:00',
   'rn': 'epRPol-mangaboul',
   'uid': '15374',
   'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a VRF to the TENANT',
  'cisco.aci.aci_vrf': {'tenant': 'nils',
   'epr_policy': 'mangaboul',
   'vrf': 'bob',
   'description': 'marcel',
   'modTs': '2024-07-05T00:55:19.682+00:00',
   'monPolDn': 'uni/tn-common/monepg-default',
   'pcEnfDirUpdated': 'yes',
   'pcTag': '32770',
   'rn': 'ctx-bob',
   'scope': '2326529',
   'seg': '2326529',
   'uid': '15374',
   'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a B

# safe version of the script, when needing a backup

In [21]:
def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', "", "::", ":all:"]

    # define exception list
    exception_list = ['aci_access_span_src_group',
                        'aci_bd',
                        'aci_bd_dhcp_label',
                        'aci_bd_subnet',
                        'aci_domain_to_vlan_pool',
                        'aci_epg_subnet',
                        'aci_fabric_span_src_group',
                        'aci_tenant_span_src_group',
                        'aci_tenant_span_src_group_src']
    
    # here define duplicates list - handle dupes in values of class mappings
    # key (class) : value (dict) -> key (parent class) : value (correct mapping)

    duplicate_list = ['spanDestGrp', 'spanSrcGrp', 'spanSrc',
                      'mgmtMaintP', 'spanRsSrcToPathEp', 'fvSubnet',
                      'fvRsPathAtt', 'vzBrCP', 'dhcpRelayP',
                      'infraRsVlanNs', 'fvRsSecInherited', 'l1PhysIf',
                      'l2extInstP', 'mgmtOoB']

    # TO DO
    # finish this mapping
    duplicate_map = {
        # 'spanDestGrp': None,
        # 'spanSrcGrp': None,
        # 'spanSrc': None,
        # 'mgmtMaintP': None,
        # 'spanRsSrcToPathEp': None,
        'fvSubnet': {'aci_epg': 'aci_epg_subnet',
                     'aci_bd': 'aci_bd_subnet'},
        # 'fvRsPathAtt': None,
        'vzBrCP': "aci_contract",
        # 'dhcpRelayP': None,
        # 'infraRsVlanNs': None,
        # 'fvRsSecInherited': None,
        # 'l1PhysIf': None,
        # 'l2extInstP': None,
        # 'mgmtOoB': None
        }
    
    def map_json_to_ansible(json_data, key, map):
        try:
            new_key = map[key]
            json_data[new_key] = json_data.pop(key) # replace with ansible term
        except(KeyError): # no match found, skip
            pass

    def remove_isNotConfigurable(key, delete_key_list):
        try:
            if isConfigurableMap[key] == False:
                delete_key_list.append(key)
            else:
                pass
        except(KeyError):
            pass

    # helper method to navigate the exception dictionary
    def map_if_duplicate(child, parent = None):
        out = None
        try:
            out = duplicate_map[child][parent]
        except(KeyError):
            out = duplicate_map[child] # means there is no exception with the parent
        return out

    # simple method to check if a key in in the exception list
    # need to run this stuff before mapping anything - mapping is ambiguous
    def isduplicate(key):
        return key in duplicate_list

    def isdefault(parent_key, key, value, map):
        try:
            return value == map[parent_key][key] # means attribute has a default value
        except(KeyError):
            return False
        
    def isexception(key): # checks if key in exceptions
        try:
            return classToAnsible[key] in exception_list
        except(KeyError):
            return False

    # def isrequired(parent_key, key, map):
    #     if key in 

    # process data recursively
    # we need to track many parent-child keys, including some of sublists etc
    def process(data, parent_key:str = None, children_parent_key:str = None, attribute_parent_key:str = None, default_map:dict = defaults, dn:dict = {}) -> dict:
        """

        Handles mapping of 
        - classes to ansible
        - class parameters to ansible

        Gets rid of the "attributes" field found in the JSON config 

        Gets rid of any fields with default values

        Handles duplicates in mappings

        """

        changes = []

        yml_tasks = [] # we will also need a list of the nested dictionaries for yml correct format

        if isinstance(data, dict):
            keys_to_delete = [] # maintain this list outside any loop, prevents "changed size" errors

            for key, value in data.items():

                # handle dn - name of parent in child, grandchild etc etc
                if isinstance(value, dict) and "children" in value.keys():
                    dn_key = reverseAliases[classToAnsible[key]]["name"][:-1]
                    dn_val = data[key]['attributes']["name"]
                    dn[dn_key] = dn_val

                # all CLASSES are handled, exceptions..
                # TO DO -> handle exceptions like "changes"
                # also not quite sure everything we are looking for is exclusively under "children" ...
                if key == "children":
                    children_parent_key = parent_key

                    # children ALWAYS contains a nested list of nested dictionaries
                    for item in value:
                        for chld_key, chld_value in item.items():

                            # the last element of each tuple in "changes" is an integer
                            # this is ONLY TO IDENTIFY WHICH TYPE OF CHANGE IS NEEDED

                            # handle duplicate mappings FIRST
                            if isduplicate(chld_key):
                                changes.append((children_parent_key, chld_key, chld_value, 0))

                # all PARAMETERS are processed and mapped if needed
                elif key == "attributes": # these are all the parameters of the classes
                    attribute_parent_key = parent_key

                    if isexception(parent_key):
                        # TYPE 1 EXCEPTIONS -> ADD PARAMS WHICH ARE FOUND IN SUBCLASSES (handled same as type2 actually)
                        try:
                            for param, path in exceptions[classToAnsible[parent_key]].items():
                                for item in data['children']: # it seems "children" is almost always found here
                                    if path[0] in item.keys():
                                        changes.append((parent_key, param, item[path[0]][path[1]][path[2]], 2))

                        except(KeyError) as e:
                            print(f"KeyError encountered: {e}")
                            pass

                    # TYPE 2 -> fix the dn attributes missing from children classes 
                    if len(dn) > 0:
                        for dn_key, dn_val in dn.items():
                            print(dn_key, dn_val)
                            changes.append((attribute_parent_key, dn_key, dn_val, 2))

                    # TYPE 2 CHANGES -> ATTRIBUTES FIELD REMOVAL, REMOVAL OF DEFAULTS
                    # append all non default params to "changes" - mapping and deletion is done here
                    for attr_key, attr_value in value.items(): # here empty values are also handled, eg if default then don't change
                        if attr_value not in default_args and not isdefault(attribute_parent_key, attr_key, attr_value, default_map):

                            # proper to fv_subnet once again? 
                            # new exception found with "ip" -> creates a mask
                            if attr_key == "ip":
                                changes.append((attribute_parent_key, 'mask', int(attr_value[-2:]), 2))

                            attr_value = attr_value if attr_key != "ip" else attr_value[:-3]

                            changes.append((attribute_parent_key, attr_key, attr_value, 2)) # appends a tuple

                            # atm hardcoding "present" into yml, change later if needed
                            changes.append((attribute_parent_key, 'state', 'present', 2))

                    keys_to_delete.append("attributes") # can append key too

                # any exception will be caught here and will crash the entire program
                # therefore we want no exceptions
                if isinstance(value, (dict, list)):
                    try:
                        process(value, key, attribute_parent_key, dn)
                    except (KeyError, TypeError) as e:
                        print(f"Error processing key {key}: {e}")
                        pass

                if isinstance(value, dict) and not value:
                    keys_to_delete.append(key)

                remove_isNotConfigurable(key, keys_to_delete) # change to boolean check? 

            # any modification to the dict can only be made outside the loops
            for key in keys_to_delete:
                del data[key]

            # use the mapping function
            for key in list(data.keys()):
                if key not in duplicate_list: # mapping done elsewhere for dupes
                    map_json_to_ansible(data, key, classToAnsible)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process(item, parent_key, attribute_parent_key, dn)

        # accessing keys in "data" instead of passing through a parent_key works
        # BECAUSE OF RECURSION, we are handling nested dictionaries as "data" everytime !!!!!!
        for change in changes:
            parent_key, child_key, child_value, change_type = change  # "change" is a 3x tuple
            if change_type == 2: # ATTRIBUTES parent key
                try:
                    new_key = reverseAliases[classToAnsible[parent_key]][child_key]
                    data[new_key] = child_value
                    if child_key in data and new_key != child_key: # UNSURE if this is necessary anymore ..
                        del data[child_key]
                except KeyError:
                    data[child_key] = child_value
    
            # DOUBLE CHECK logic here, otherwise seems to work as expected
            # also can get rid of the whole "CHILDREN" key and move all upwards when we need it, only issue is how does this translate in yml
            # FOR NOW keep "children" key as it helps with ordering everything correctly
            elif change_type == 0: # CHILDREN parent key, DUPLICATE CASES
                try:
                    new_key = map_if_duplicate(child_key, classToAnsible[parent_key])
                    data['children'][0][new_key] = child_value
                    if child_key in data['children'][0]: # is this always the case? 0 idx? double check logic
                        del data['children'][0][child_key]
                except(KeyError):
                    pass

        return data

    # this function gets rid of empty structures in the output, think of {} for example
    # code is from GPT
    def remove_empty_dicts(data):
        if isinstance(data, dict):
            return {k: remove_empty_dicts(v) for k, v in data.items() if remove_empty_dicts(v)} # see return of function for why this works
        elif isinstance(data, list):
            for idx, item in enumerate(data):
                if isinstance(item, dict) and len(item) == 0:
                    data.pop(idx)
                else:
                    data[idx] = remove_empty_dicts(item)
            return data
        else:
            return data # this basically evaluates to false when the structure is None

    # process the initial data
    a = process(data)

    # delete the empty dictionaries
    b = remove_empty_dicts(a)

    return data

# regex testing 

In [22]:
import os
import yaml

class YAMLItem:
    def __init__(self, key, value, parent=None):
        self.key = key
        self.value = value
        self.parent = parent
        self.children = []

        if isinstance(value, dict):
            for k, v in value.items():
                self.children.append(YAMLItem(k, v, parent=self))
        elif isinstance(value, list):
            for v in value:
                self.children.append(YAMLItem(None, v, parent=self))

    def is_duplicate(self, duplicate_list):
        return self.key in duplicate_list

    def is_default(self, default_map, default_args):
        if self.parent and isinstance(self.parent.value, dict):
            try:
                return self.value == default_map[self.parent.key][self.key] or self.value in default_args
            except KeyError:
                return False
        return False

    def is_exception(self, class_to_ansible, exception_list):
        try:
            return class_to_ansible[self.key] in exception_list
        except KeyError:
            return False

    def map_key(self, class_to_ansible, reverse_aliases):
        if self.key in class_to_ansible:
            self.key = class_to_ansible[self.key]

    def remove_empty_dicts(self):
        if isinstance(self.value, dict):
            self.value = {k: v for k, v in self.value.items() if v}
            for child in self.children:
                child.remove_empty_dicts()

        elif isinstance(self.value, list):
            self.value = [v for v in self.value if v]
            for child in self.children:
                child.remove_empty_dicts()

    def reconstruct(self):
        if isinstance(self.value, dict):
            self.value = {child.key: child.value for child in self.children}

    def to_dict(self):
        if self.key:
            return {self.key: self.value}
        return self.value

class YAMLProcessor:
    def __init__(self, data, out_dir=None):
        self.data = data
        self.out_dir = out_dir or os.getcwd()
        self.default_args = ['', "", "::", ":all:"] # adapt if more exceptions are found
        self.exception_list = [
            'aci_access_span_src_group', 'aci_bd', 'aci_bd_dhcp_label', 'aci_bd_subnet',
            'aci_domain_to_vlan_pool', 'aci_epg_subnet', 'aci_fabric_span_src_group',
            'aci_tenant_span_src_group', 'aci_tenant_span_src_group_src'
        ]

        self.duplicate_list = [
            'spanDestGrp', 'spanSrcGrp', 'spanSrc', 'mgmtMaintP', 'spanRsSrcToPathEp',
            'fvSubnet', 'fvRsPathAtt', 'vzBrCP', 'dhcpRelayP', 'infraRsVlanNs',
            'fvRsSecInherited', 'l1PhysIf', 'l2extInstP', 'mgmtOoB'
        ]

        self.duplicate_map = { # need to complete this one with all the duplicates at some stage
            'fvSubnet': {'aci_epg': 'aci_epg_subnet', 'aci_bd': 'aci_bd_subnet'},
            'vzBrCP': "aci_contract"
        }

        self.class_to_ansible = classToAnsible
        self.reverse_aliases = reverseAliases

    def map_if_duplicate(self, child, parent=None):
        try:
            return self.duplicate_map[child][parent]
        except KeyError:
            return self.duplicate_map.get(child, child)

    def process_item(self, item):
        if isinstance(item.value, dict):
            for child in item.children:
                self.process_item(child)

    def remove_empty_dicts(self, item):
        item.remove_empty_dicts()

    def reconstruct(self, item):
        item.reconstruct()

    def to_dict(self, item):
        return item.to_dict()

    def save_to_yaml(self, data):
        save_path = os.path.join(self.out_dir, "ansible_reconstructed.yml")
        with open(save_path, 'w') as file:
            yaml.dump(data, file, default_flow_style=False)
        print(f"YAML file has been saved to {save_path}")

    def process(self):
        root_item = YAMLItem(None, self.data)
        self.process_item(root_item)
        self.remove_empty_dicts(root_item)
        self.reconstruct(root_item)
        data = self.to_dict(root_item)
        self.save_to_yaml(data)
        return data

In [141]:
import os
import json
from mappings import classToAnsible, ansibleToClass # need to get the mapping to ansible from json
import yaml
import pprint
from isConfigurableMap import isConfigurableMap # need to get the 
from defaultClassAttrValues import defaults
from exceptionDict import exceptions
from requiredParamsAliasesMap import requiredParamsAliases, reverse_alias_map

# define all the paths here
PATH_TO_SAVE = "ansible-reconstructed.yml"
PATH_TO_JSON = "tn-dev-test.json"
PATH_TO_CREDENTIALS = ""

### start of the main function ###
reverseAliases = reverse_alias_map(requiredParamsAliases)

# this will be refactored at some stage, maybe some OOP or just cleaned, for now it does what I need it to

def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', "", "::", ":all:"]

    # invisible_arguments
    invisible_args = ["annotation", "dn", "rn", "uid", "modTs", "monPolDn", "seg", "pcTag", "userdom", "tDn", "filter_nam"] # adjust as needed

    # define exception list
    exception_list = ['aci_access_span_src_group',
                        'aci_bd',
                        'aci_bd_dhcp_label',
                        'aci_bd_subnet',
                        'aci_domain_to_vlan_pool',
                        'aci_epg_subnet',
                        'aci_fabric_span_src_group',
                        'aci_tenant_span_src_group',
                        'aci_tenant_span_src_group_src']
    
    # here define duplicates list - handle dupes in values of class mappings
    # key (class) : value (dict) -> key (parent class) : value (correct mapping)

    duplicate_list = ['spanDestGrp', 'spanSrcGrp', 'spanSrc',
                      'mgmtMaintP', 'spanRsSrcToPathEp', 'fvSubnet',
                      'fvRsPathAtt', 'vzBrCP', 'dhcpRelayP',
                      'infraRsVlanNs', 'fvRsSecInherited', 'l1PhysIf',
                      'l2extInstP', 'mgmtOoB']

    # TO DO
    # finish this mapping
    duplicate_map = {
        # 'spanDestGrp': None,
        # 'spanSrcGrp': None,
        # 'spanSrc': None,
        # 'mgmtMaintP': None,
        # 'spanRsSrcToPathEp': None,
        'fvSubnet': {'aci_epg': 'aci_epg_subnet',
                     'aci_bd': 'aci_bd_subnet'},
        # 'fvRsPathAtt': None,
        'vzBrCP': "aci_contract",
        # 'dhcpRelayP': None,
        # 'infraRsVlanNs': None,
        # 'fvRsSecInherited': None,
        # 'l1PhysIf': None,
        # 'l2extInstP': None,
        # 'mgmtOoB': None
        }
    
    def map_json_to_ansible(json_data, key, map):
        try:
            new_key = map[key]
            json_data[new_key] = json_data.pop(key) # replace with ansible term
        except(KeyError): # no match found, skip
            pass

    def remove_isNotConfigurable(key, delete_key_list):
        try:
            if isConfigurableMap[key] == False:
                delete_key_list.append(key)
            else:
                pass
        except(KeyError):
            pass

    # helper method to navigate the exception dictionary
    def map_if_duplicate(child, parent = None):
        out = None
        try:
            out = duplicate_map[child][parent]
        except(KeyError):
            out = duplicate_map[child] # means there is no exception with the parent
        return out

    # simple method to check if a key in in the exception list
    # need to run this stuff before mapping anything - mapping is ambiguous
    def isduplicate(key):
        return key in duplicate_list

    def isdefault(parent_key, key, value, map):
        try:
            return value == map[parent_key][key] # means attribute has a default value
        except(KeyError):
            return False
        
    def isexception(key): # checks if key in exceptions
        try:
            return classToAnsible[key] in exception_list
        except(KeyError):
            return False

    def isfullydefault(val, parent_key, default_map, second_default_map):
        try:
            # convert defaults to sets 
            default_values_set = set(default_map[parent_key].values())
            second_default_values_set = set(second_default_map)
            
            # combine default args and default mapping
            combined_default_values_set = default_values_set.union(second_default_values_set)
            
            # to set 
            val_set = set(val)
            
            # check if subset
            return val_set.issubset(combined_default_values_set)
        except KeyError: # anything which is not "fv" is here atm
            return True # change based on desired behavior
        
    def save_to_yaml(save_path, data):
        # save_path = os.path.join(save_path, "ansible_reconstructed.yml")
        with open("ansible_reconstructed.yml", 'w') as file:
            yaml.dump(a, file, default_flow_style = False, sort_keys = False)
        print(f"YAML file has been saved to ansible_reconstructed.yml")

    # process data recursively
    # we need to track many parent-child keys, including some of sublists etc
    def process(data,
                parent_key:str = None,
                grandparent_key:str = None,
                default_map:dict = defaults,
                dn:dict = {},
                dn_key_stack:list = []) -> dict:
        """

        Handles mapping of 
        - classes to ansible
        - class parameters to ansible

        Gets rid of the "attributes" field found in the JSON config 

        Gets rid of any fields with default values

        Handles duplicates in mappings

        """

        changes = []

        if isinstance(data, dict):
            keys_to_delete = [] # maintain this list outside any loop, prevents "changed size" errors

            for key, value in data.items():

                # all CLASSES are handled, exceptions..
                # TO DO -> handle exceptions like "changes"
                # also not quite sure everything we are looking for is exclusively under "children" ...
                if key == "children":

                    # children ALWAYS contains a nested list of nested dictionaries
                    for item in value:
                        for chld_key, chld_value in item.items():

                            # the last element of each tuple in "changes" is an integer
                            # this is ONLY TO IDENTIFY WHICH TYPE OF CHANGE IS NEEDED

                            # handle duplicate mappings FIRST
                            if isduplicate(chld_key):
                                changes.append((parent_key, chld_key, chld_value, 0))

                # all PARAMETERS are processed and mapped if needed
                elif key == "attributes": # these are all the parameters of the classes

                    # check if all attributes are default; if so, skip processing entirely
                    # empty dictionaries function will take care of it
                    if not isfullydefault(value.values(), parent_key, defaults, default_args):

                        if isexception(parent_key):

                            # TYPE 1 EXCEPTIONS -> ADD PARAMS WHICH ARE FOUND IN SUBCLASSES (handled same as type2 actually)
                            try:
                                for param, path in exceptions[classToAnsible[parent_key]].items():
                                    for item in data['children']: # it seems "children" is almost always found here
                                        if path[0] in item.keys():
                                            changes.append((parent_key, param, item[path[0]][path[1]][path[2]], 2))

                                            #######
                                            # path[0] is the exception class we need to get rid of, maybe through the changes system
                                            #######

                            except(KeyError) as e:
                                print(f"KeyError encountered: {e}")
                                pass

                        # TYPE 2 CHANGES -> ATTRIBUTES FIELD REMOVAL, REMOVAL OF DEFAULTS
                        # append all non default params to "changes" - mapping and deletion is done here
                        for attr_key, attr_value in value.items(): # here empty values are also handled, eg if default then don't change
                            if attr_key not in invisible_args and attr_value not in default_args and not isdefault(parent_key, attr_key, attr_value, default_map):

                                # proper to fv_subnet once again?
                                # new exception found with "ip" -> creates a mask
                                if attr_key == "ip":
                                    attr_key = "gateway"
                                    changes.append((parent_key, 'mask', int(attr_value[-2:]), 2))

                                attr_value = attr_value if attr_key != "gateway" else attr_value[:-3]

                                changes.append((parent_key, attr_key, attr_value, 2)) # appends a tuple

                        # atm hardcoding "present" into yml, change later if needed
                        changes.append((parent_key, 'state', 'present', 2))

                    keys_to_delete.append("attributes") # can append key too

                # any exception will be caught here and will crash the entire program
                # therefore we want no exceptions
                if isinstance(value, (dict, list)):
                    try:
                        process(data = value, parent_key = key, grandparent_key = parent_key)
    
                    except (KeyError) as e:
                        print(f"Error processing key {key}: {e}")
                        pass

                if isinstance(value, dict) and not value:
                    keys_to_delete.append(key)

                remove_isNotConfigurable(key, keys_to_delete) # change to boolean check? 

            # any modification to the dict can only be made outside the loops
            for key in keys_to_delete:
                try:
                    del data[key]
                except(KeyError):
                    pass

            # use the mapping function
            for key in list(data.keys()):
                if key not in duplicate_list: # mapping done elsewhere for dupes
                    map_json_to_ansible(data, key, classToAnsible)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process(data = item, parent_key = parent_key, grandparent_key = grandparent_key)

        # accessing keys in "data" instead of passing through a parent_key works
        # BECAUSE OF RECURSION, we are handling nested dictionaries as "data" everytime !!!!!!
        for change in changes:
            parent_key, child_key, child_value, change_type = change  # "change" is a 3x tuple
            if change_type == 2: # ATTRIBUTES parent key
                try:
                    new_key = reverseAliases[classToAnsible[parent_key]][child_key]

                    # at the moment no use for the required tag, can change later on if we need it somehow
                    if new_key[-1] == "*":
                        new_key = new_key[:-1]

                    data[new_key] = child_value
                    if child_key in data and new_key != child_key: # UNSURE if this is necessary anymore ..
                        del data[child_key]
                except KeyError:
                    data[child_key] = child_value

            # DOUBLE CHECK logic here, otherwise seems to work as expected
            # also can get rid of the whole "CHILDREN" key and move all upwards when we need it, only issue is how does this translate in yml
            # FOR NOW keep "children" key as it helps with ordering everything correctly
            elif change_type == 0: # CHILDREN parent key, DUPLICATE CASES
                try:
                    new_key = map_if_duplicate(child_key, classToAnsible[parent_key])
                    data['children'][0][new_key] = child_value
                    if child_key in data['children'][0]: # is this always the case? 0 idx? double check logic
                        del data['children'][0][child_key]
                except(KeyError):
                    pass

        return data

    # this function gets rid of empty structures in the output, think of {} for example
    # code is from GPT
    def remove_empty_dicts(data):
        if isinstance(data, dict):
            return {k: remove_empty_dicts(v) for k, v in data.items() if remove_empty_dicts(v)} # see return of function for why this works
        elif isinstance(data, list):
            for idx, item in enumerate(data):
                if isinstance(item, dict) and len(item) == 0:
                    data.pop(idx)
                else:
                    data[idx] = remove_empty_dicts(item)
            return data
        else:
            return data # this basically evaluates to false when the structure is None

    # this function rebuilds the DN attributes, when parent attributes are needed etc
    # cannot figure out way to fix this withing the loop atm, this does the trick just fine
    # def build_dn(data, ancestors=[], parent_child_map = {}):
    #     if isinstance(data, dict):
    #         for key, value in data.items():
    #             if key != 'children' and isinstance(value, dict):
    #                 if key not in parent_child_map:
    #                     parent_child_map[key] = []
    #                 parent_child_map[key].extend(ancestors)
    #                 build_dn(value, ancestors + [key])
    #             elif key == 'children':
    #                 for child in value:
    #                     build_dn(child, ancestors)

    #     return parent_child_map

    # this function gathers all hierarchical parameter names - further used in reconstruct function
    # returns a dictionary with list values and string keys
    def get_parent_attributes(data, 
                          parent_key=None, 
                          grandparent_key=None, 
                          great_grandparent_key=None, 
                          great_great_grandparent_key=None, 
                          parent_name=None, 
                          grandparent_name=None, 
                          great_grandparent_name=None, 
                          great_great_grandparent_name=None, 
                          changes=None):

        if changes is None:
            changes = {}

        if isinstance(data, dict):
            for key, value in data.items():
                try:
                    current_name = reverseAliases[key]["name"][:-1]  # remove mandatory tag
                except KeyError:
                    current_name = None

                if parent_key == "children": # need to append all this info for later use - iterate over pairwise, will work just fine
                    change = [i for i in [parent_name, grandparent_name, great_grandparent_name, great_great_grandparent_name, grandparent_key, great_great_grandparent_key] if i is not None]
                    changes[key] = change

                # could make it more clear somehow.. 
                get_parent_attributes(value, key, parent_key, grandparent_key, great_grandparent_key, 
                                    current_name, parent_name, grandparent_name, great_grandparent_name, changes)

        elif isinstance(data, list):
            for item in data:
                get_parent_attributes(item, parent_key, grandparent_key, great_grandparent_key, great_great_grandparent_key, 
                                    parent_name, grandparent_name, great_grandparent_name, great_great_grandparent_name, changes)

        return changes

    # handle reconstruction of yml with recursion
    # TO DO -> ADD REMVOVAL OF "fvRs" CLASSES, too hard to handle above due to recursive issues
    def rebuild_yml(data, credentials_file = None, yml_list = [], parent_key = None, grandparent_key = None, dn_key_stack = [], dn = {}):

        entry_dict = {}

        if isinstance(data, dict):
            for key, value in data.items():

                nested_dictionary = {}

                # here restructuring is handled
                if parent_key == "children" or parent_key == None:

                    # bad trick to get rid of all non-aci modules
                    if key[:3] == "aci":
                        pass
                    else:
                        break

                    # set name to something generic
                    key_name = key.split("_")[1] if len(key.split("_")) == 2 else key.split("_")[2]
                    if parent_key != None:
                        grandparent_key_name = grandparent_key.split("_")[1] if len(grandparent_key.split("_")) == 2 else grandparent_key.split("_")[2:]
                        sentence = f"add a {key_name.upper()} to the {grandparent_key_name.upper()}"
                    else:
                        sentence = f"create {key_name} on ACI using {key} module"

                    entry_dict["name"] = sentence
                    for subkey, subvalue in data[key].items():
                        if subkey != "children":
                            # print(key, subkey, subvalue)
                            nested_dictionary[subkey] = subvalue
                    entry_dict["cisco.aci." + key] = nested_dictionary
                    entry_dict["delegate_to"] = "localhost"

                    yml_list.append(entry_dict)

                if isinstance(value, dict):
                    rebuild_yml(value, parent_key = key, grandparent_key = parent_key)

                elif isinstance(value, list):
                    for item in value:
                        rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

        elif isinstance(data, list):
            for item in data:
                rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

        return yml_list

    # process the initial data
    a = process(data)

    # delete the empty dictionaries
    b = remove_empty_dicts(a)

    c = get_parent_attributes(b)

    fin = rebuild_yml(b)

    return b

if __name__ == "__main__":
    
    with open(PATH_TO_JSON, 'r') as file:
        y = json.load(file)

    out = reconstruct_yml(y)

    # save_path = os.path.join(save_path, "ansible_reconstructed.yml")
    with open(PATH_TO_SAVE, 'w') as file:
        yaml.dump(out, file, default_flow_style = False, sort_keys = False)

    print(f"YAML file has been saved to {PATH_TO_SAVE}")


KeyError encountered: 'children'
KeyError encountered: 'children'
KeyError encountered: 'children'
KeyError encountered: 'children'
YAML file has been saved to ansible-reconstructed.yml


In [142]:
out

{'aci_tenant': {'children': [{'aci_vrf': {'vrf': 'dev.dev-vrf',
     'state': 'present'}},
   {'aci_bd': {'children': [{'aci_bd_subnet': {'mask': 24,
        'gateway': '10.1.10.1',
        'state': 'present'}},
      {'fvRsCtx': {'tnFvCtxName': 'dev.dev-vrf', 'state': 'present'}}],
     'endpoint_retention_action': 'resolve',
     'vrf': 'dev.dev-vrf',
     'arp_flooding': 'yes',
     'enable_rogue_except_mac': 'no',
     'mac_address': '00:22:BD:F8:19:FF',
     'bd': '10.1.10.0_24',
     'l2_unknown_unicast': 'flood',
     'state': 'present'}},
   {'aci_bd': {'children': [{'aci_bd_subnet': {'mask': 24,
        'gateway': '10.1.13.1',
        'state': 'present'}},
      {'fvRsCtx': {'tnFvCtxName': 'dev.dev-vrf', 'state': 'present'}}],
     'endpoint_retention_action': 'resolve',
     'vrf': 'dev.dev-vrf',
     'arp_flooding': 'yes',
     'enable_rogue_except_mac': 'no',
     'mac_address': '00:22:BD:F8:19:FF',
     'bd': '10.1.13.0_24',
     'l2_unknown_unicast': 'flood',
     'state'

In [25]:
def merge_siblings(data, merge = {}):
    yml_list_map = {} # save all mappings here, map outside main loop (or change during iteration error)
    variation_list = []
    nested_dict = {}
    nested_key = None

    for module in data:
        aci_class = list(module.keys())[1].split(".")[2] # aci_tenant, aci_vrf, aci_bd ...

        if aci_class in merge:
            for subkey, subvalue in module.items():
                if subkey.split(".")[0] == "cisco": # only go in attributes
                    for subsubkey, subsubvalue in subvalue.items():
                        try: # id and map merge
                            if subsubvalue != comparable[subsubkey]:
                                # print("change detected : ", subsubkey, subsubvalue, comparable[subsubkey])
                                if comparable[subsubkey] not in variation_list:
                                    variation_list.append(comparable[subsubkey])
                                    nested_key = subsubkey
                                variation_list.append(subsubvalue)
                        except(KeyError):
                            pass

            print(aci_class)
            print(variation_list, nested_key)

            if nested_key: # not None
                nested_dict[nested_key] = variation_list.copy()  # Use .copy() to avoid mutable reference issues

                print(nested_dict)

                yml_list_map[aci_class] = nested_dict.copy()  # Use .copy() to avoid mutable reference issues

                print("\n")

        # in this block we simply are in another class - say aci_bd -> aci_epg
        # therefore we need to reset everything
        if aci_class not in merge:
            merge[aci_class] = module
            comparable = merge[aci_class]["cisco.aci." + aci_class]
            variation_list = []  # Create a new list
            nested_dict = {}  # Create a new dictionary
            nested_key = None


        # TO DO 
        # apply all changes (build the loop)

    return yml_list_map

In [26]:
merge_siblings(out)

aci_bd
[] None
aci_bd_subnet
['10.1.10.1', '10.1.13.1'] gateway
{'gateway': ['10.1.10.1', '10.1.13.1']}


aci_bd
['10.1.10.1', '10.1.13.1'] gateway
{'gateway': ['10.1.10.1', '10.1.13.1']}


aci_bd_subnet
['10.1.10.1', '10.1.13.1', '10.1.11.1'] gateway
{'gateway': ['10.1.10.1', '10.1.13.1', '10.1.11.1']}


aci_bd
['10.1.10.1', '10.1.13.1', '10.1.11.1'] gateway
{'gateway': ['10.1.10.1', '10.1.13.1', '10.1.11.1']}


aci_bd_subnet
['10.1.10.1', '10.1.13.1', '10.1.11.1', '10.1.12.1'] gateway
{'gateway': ['10.1.10.1', '10.1.13.1', '10.1.11.1', '10.1.12.1']}


aci_epg
[] None
aci_epg_to_domain
['vlan-2027', 'vlan-2023', 'vlan-2026', 'vlan-2022'] primary_encap
{'primary_encap': ['vlan-2027', 'vlan-2023', 'vlan-2026', 'vlan-2022']}


aci_epg
['vlan-2027', 'vlan-2023', 'vlan-2026', 'vlan-2022'] primary_encap
{'primary_encap': ['vlan-2027', 'vlan-2023', 'vlan-2026', 'vlan-2022']}


aci_epg_to_domain
['vlan-2027', 'vlan-2023', 'vlan-2026', 'vlan-2022', 'vlan-2021', 'vlan-2020'] primary_encap
{'pri

{'aci_bd_subnet': {'gateway': ['10.1.10.1',
   '10.1.13.1',
   '10.1.11.1',
   '10.1.12.1']},
 'aci_bd': {'gateway': ['10.1.10.1', '10.1.13.1', '10.1.11.1']},
 'aci_epg_to_domain': {'primary_encap': ['vlan-2027',
   'vlan-2023',
   'vlan-2026',
   'vlan-2022',
   'vlan-2021',
   'vlan-2020',
   'vlan-2025',
   'vlan-2024']},
 'aci_epg': {'primary_encap': ['vlan-2027',
   'vlan-2023',
   'vlan-2026',
   'vlan-2022',
   'vlan-2021',
   'vlan-2020']}}

In [30]:
out

{'aci_tenant': {'children': [{'aci_vrf': {'vrf': 'dev.dev-vrf',
     'state': 'present'}},
   {'aci_bd': {'children': [{'aci_bd_subnet': {'mask': 24,
        'gateway': '10.1.10.1',
        'state': 'present'}},
      {'fvRsCtx': {'tnFvCtxName': 'dev.dev-vrf', 'state': 'present'}}],
     'endpoint_retention_action': 'resolve',
     'vrf': 'dev.dev-vrf',
     'arp_flooding': 'yes',
     'enable_rogue_except_mac': 'no',
     'mac_address': '00:22:BD:F8:19:FF',
     'bd': '10.1.10.0_24',
     'l2_unknown_unicast': 'flood',
     'state': 'present'}},
   {'aci_bd': {'children': [{'aci_bd_subnet': {'mask': 24,
        'gateway': '10.1.13.1',
        'state': 'present'}},
      {'fvRsCtx': {'tnFvCtxName': 'dev.dev-vrf', 'state': 'present'}}],
     'endpoint_retention_action': 'resolve',
     'vrf': 'dev.dev-vrf',
     'arp_flooding': 'yes',
     'enable_rogue_except_mac': 'no',
     'mac_address': '00:22:BD:F8:19:FF',
     'bd': '10.1.13.0_24',
     'l2_unknown_unicast': 'flood',
     'state'

In [151]:
def get_parent_attributes(data, 
                          parent_key=None, 
                          grandparent_key=None, 
                          great_grandparent_key=None, 
                          great_great_grandparent_key=None, 
                          parent_name=None, 
                          grandparent_name=None, 
                          great_grandparent_name=None, 
                          great_great_grandparent_name=None, 
                          changes=None):

    if changes is None:
        changes = {}

    if isinstance(data, dict):
        for key, value in data.items():
            try:
                current_name = reverseAliases[key]["name"][:-1]  # remove mandatory tag
            except KeyError:
                current_name = None

            if parent_key == "children": # need to append all this info for later use - iterate over pairwise, will work just fine
                change = [i for i in [parent_name, grandparent_name, great_grandparent_name, great_great_grandparent_name, grandparent_key, great_great_grandparent_key] if i is not None]
                changes[key] = change

            # could make it more clear somehow.. 
            get_parent_attributes(value, key, parent_key, grandparent_key, great_grandparent_key, 
                                  current_name, parent_name, grandparent_name, great_grandparent_name, changes)

    elif isinstance(data, list):
        for item in data:
            get_parent_attributes(item, parent_key, grandparent_key, great_grandparent_key, great_great_grandparent_key, 
                                  parent_name, grandparent_name, great_grandparent_name, great_great_grandparent_name, changes)

    return changes

In [173]:
mappp = get_parent_attributes(out)

In [176]:
mappp

{'aci_vrf': ['tenant', 'aci_tenant'],
 'aci_bd': ['tenant', 'aci_tenant'],
 'aci_bd_subnet': ['bd', 'tenant', 'aci_bd', 'aci_tenant'],
 'fvRsCtx': ['bd', 'tenant', 'aci_bd', 'aci_tenant'],
 'aci_ap': ['tenant', 'aci_tenant'],
 'aci_esg': ['ap', 'tenant', 'aci_ap', 'aci_tenant'],
 'fvRsScope': ['esg', 'ap', 'aci_esg', 'aci_ap'],
 'aci_epg_to_contract_interface': ['esg', 'ap', 'aci_esg', 'aci_ap'],
 'fvEPgSelector': ['esg', 'ap', 'aci_esg', 'aci_ap'],
 'aci_epg': ['ap', 'tenant', 'aci_ap', 'aci_tenant'],
 'aci_epg_to_domain': ['epg', 'ap', 'aci_epg', 'aci_ap'],
 'fvRsBd': ['epg', 'ap', 'aci_epg', 'aci_ap']}

In [177]:
def rebuild_yml(data, dn_attributes_map = mappp, credentials_file = None, yml_list = [], parent_key = None, grandparent_key = None):

        entry_dict = {}

        if isinstance(data, dict):
            for key, value in data.items():

                nested_dictionary = {}

                # here restructuring is handled
                if parent_key == "children" or parent_key == None:

                    # bad trick to get rid of all non-aci modules
                    if key[:3] == "aci":
                        pass
                    else:
                        break

                    # set name to something generic
                    key_name = key.split("_")[1] if len(key.split("_")) == 2 else key.split("_")[2]

                    if parent_key != None:
                        grandparent_key_name = grandparent_key.split("_")[1] if len(grandparent_key.split("_")) == 2 else grandparent_key.split("_")[2:]
                        sentence = f"add a {key_name.upper()} to the {grandparent_key_name.upper()}"
                    else:
                        sentence = f"create {key_name} on ACI using {key} module"

                    entry_dict["name"] = sentence
                    for subkey, subvalue in data[key].items():
                        if subkey != "children":
                            # print(key, subkey, subvalue)
                            nested_dictionary[subkey] = subvalue


                    if key in dn_attributes_map:
                        print(dn_attributes_map[key])


                    
                    entry_dict["cisco.aci." + key] = nested_dictionary
                    entry_dict["delegate_to"] = "localhost"

                    yml_list.append(entry_dict)

                if isinstance(value, dict):
                    rebuild_yml(value, parent_key = key, grandparent_key = parent_key)

                elif isinstance(value, list):
                    for item in value:
                        rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

        elif isinstance(data, list):
            for item in data:
                rebuild_yml(item, parent_key = key, grandparent_key = parent_key)

        return yml_list

In [178]:
rebuild_yml(out)

['tenant', 'aci_tenant']
['tenant', 'aci_tenant']
['bd', 'tenant', 'aci_bd', 'aci_tenant']
['tenant', 'aci_tenant']
['bd', 'tenant', 'aci_bd', 'aci_tenant']
['tenant', 'aci_tenant']
['bd', 'tenant', 'aci_bd', 'aci_tenant']
['tenant', 'aci_tenant']
['bd', 'tenant', 'aci_bd', 'aci_tenant']
['tenant', 'aci_tenant']
['ap', 'tenant', 'aci_ap', 'aci_tenant']
['esg', 'ap', 'aci_esg', 'aci_ap']
['ap', 'tenant', 'aci_ap', 'aci_tenant']
['epg', 'ap', 'aci_epg', 'aci_ap']
['ap', 'tenant', 'aci_ap', 'aci_tenant']
['epg', 'ap', 'aci_epg', 'aci_ap']
['ap', 'tenant', 'aci_ap', 'aci_tenant']
['epg', 'ap', 'aci_epg', 'aci_ap']
['ap', 'tenant', 'aci_ap', 'aci_tenant']
['epg', 'ap', 'aci_epg', 'aci_ap']


[{'name': 'create tenant on ACI using aci_tenant module',
  'cisco.aci.aci_tenant': {'tenant': 'dev_mart', 'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a VRF to the TENANT',
  'cisco.aci.aci_vrf': {'vrf': 'dev.dev-vrf', 'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a BD to the TENANT',
  'cisco.aci.aci_bd': {'endpoint_retention_action': 'resolve',
   'vrf': 'dev.dev-vrf',
   'arp_flooding': 'yes',
   'enable_rogue_except_mac': 'no',
   'mac_address': '00:22:BD:F8:19:FF',
   'bd': '10.1.10.0_24',
   'l2_unknown_unicast': 'flood',
   'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a SUBNET to the BD',
  'cisco.aci.aci_bd_subnet': {'mask': 24,
   'gateway': '10.1.10.1',
   'state': 'present'},
  'delegate_to': 'localhost'},
 {'name': 'add a BD to the TENANT',
  'cisco.aci.aci_bd': {'endpoint_retention_action': 'resolve',
   'vrf': 'dev.dev-vrf',
   'arp_flooding': 'yes',
   'enable_rogue_except_mac': 'no',
   'mac_address': '00

In [163]:
for module in spe:
    for key, value in module.items():
        if isinstance(value, dict):
            print(key)

cisco.aci.aci_tenant
cisco.aci.aci_vrf
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_ap
cisco.aci.aci_esg
cisco.aci.aci_epg_to_contract_interface
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_tenant
cisco.aci.aci_vrf
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_bd
cisco.aci.aci_bd_subnet
cisco.aci.aci_ap
cisco.aci.aci_esg
cisco.aci.aci_epg_to_contract_interface
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
cisco.aci.aci_epg
cisco.aci.aci_epg_to_domain
